In [21]:
import pandas as pd
import news_df
import soup
import news_dict
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import cookielib
import urllib2
import requests
import re

In [19]:
def get_soup(url):
    #request = urllib2.Request(url)
    #response = urllib2.urlopen(url) #request
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html.decode('utf-8', 'ignore'), 'html.parser')
    results = soup.find_all('div', attrs = {'class': 'story-body'})
    return results

In [24]:
#!/usr/bin/env python

def scrape_soup(results):

    # initialize lists
    titles = []
    dates = []
    links = []
    full_texts = []
    authors = []
    sections = []

    # scrape results into lists
    for x in results:
        # get link
        link = x.find('a')['href']
        # set regex to eliminate interactive features
        match = re.search('^http://www.nytimes.com/20', link)
        if match:
            links.append(link)

    # resoup it
    for link in links:
        all_p = ''
        cj = cookielib.CookieJar()
        opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
        new_soup = BeautifulSoup(opener.open(link).read().decode('utf-8', 'ignore'), 'html.parser')
        # get the article content
        body = new_soup.find_all('p', attrs = {'class': 'story-body-text story-content'})
        for p in body:
            new_p = p.text.strip()
            all_p = all_p + new_p
        full_texts.append(all_p)

        # get titles
        title = new_soup.find('meta', attrs = {'property': 'og:title'})['content']
        titles.append(title)

        # get authors
        author = new_soup.find('meta', attrs = {'name': 'author'})['content']
        authors.append(author)

        # get sections
        section = new_soup.find('meta', attrs = {'name': 'CG'})['content']
        sections.append(section)

        # get dates
        date = new_soup.find('meta', attrs = {'name': 'pdate'})['content']
        dates.append(date)

    data_dict = {
        'title': titles, 'link': links, 'author': authors, 'body': full_texts,
        'section': sections, 'date': dates
    }
    return data_dict


In [25]:
def make_df(data_dict):
    df = pd.DataFrame(data_dict)
    return df

In [ ]:
# should make a function that takes row by row from dataframe and inserts into ny_times table if it doesn't already exist to prevent duplicates


In [26]:
def get_news(url):
    engine = create_engine('postgresql://teresaborcuch@localhost:5433/capstone')
    my_soup = get_soup(url)
    my_news_dict = scrape_soup(my_soup)
    df = make_df(my_news_dict)
    df.to_sql('nytimes', engine, if_exists = 'append', index = False)

In [41]:
get_news('http://www.nytimes.com/section/world')

In [42]:
get_news('http://www.nytimes.com/section/health')

In [43]:
get_news('http://www.nytimes.com/section/technology')

In [44]:
get_news('http://www.nytimes.com/section/us')

In [45]:
get_news('http://www.nytimes.com/section/opinion')

In [46]:
get_news('http://www.nytimes.com/section/science')

In [47]:
get_news('http://www.nytimes.com/section/business')

In [48]:
get_news('http://www.nytimes.com/section/arts')

In [49]:
get_news('http://www.nytimes.com/section/sports')